In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dados-lanchonete-pao/DataBase po.xlsx


In [2]:
dados_pao = pd.read_excel("/kaggle/input/dados-lanchonete-pao/DataBase po.xlsx")

In [3]:
dados_pao_copia = dados_pao.copy()
dados_pao_copia.columns = [col.strip().lower().replace(' ', '_') for col in dados_pao_copia.columns]

In [4]:
dados_pao_copia['sexo'] = dados_pao_copia['sexo'].replace({'masculino': 'M', 'feminino': 'F'})
dados_pao_copia['idade'] = dados_pao_copia['idade'].fillna(dados_pao_copia['idade'].median())
dados_pao_copia['pão_comprados'] = dados_pao_copia['pão_comprados'].fillna(0)
dados_pao_copia['valor_da_ultima_compra'] = dados_pao_copia['valor_da_ultima_compra'].fillna(0)

dados_pao_copia['frequencia'] = dados_pao_copia['frequencia'].fillna('Semanal')
dados_pao_copia['trabalha'] = dados_pao_copia['trabalha'].fillna('Não')
dados_pao_copia['faltas'] = dados_pao_copia['faltas'].fillna(0)

# estado civil, número de amigos

In [5]:
# Converter valores textuais para NaN e manter apenas valores numéricos
#    dados_pao['interação_social'] = pd.to_numeric(dados_pao_copia['interação_social'], errors='coerce')
#    median = dados_pao['interação_social'].mean()
#    maximo = dados_pao['interação_social'].max()
    
#    alta = maximo - median

#def categorizar_interacao(valor):
#    if valor == "Alta":
 #       return alta
#    elif valor == "Média":
 #       return 
#    elif valor >= limite_media:
 #       return 'Média'
#    else:
#        return 'Baixa'

#dados_pao_copia['interação_social_categorizada'] = dados_pao_copia['interação_social'].apply(categorizar_interacao)

In [6]:
# Classificação dos clientes
def classificar_cliente(row):
    pontos = 0
    if row['frequencia'] == 'Semanal':
        pontos += 1
    elif row['frequencia'] == 'Quinzenal':
        pontos += 0.5
    elif row['frequencia'] == 'Mensal':
        pontos += 0.25
    
    if row['pão_comprados'] > 5:
        pontos += 1
    elif 3 <= row['pão_comprados'] <= 5:
        pontos += 0.5
    elif row['pão_comprados'] < 3:
        pontos += 0.25
    
    if row['valor_da_ultima_compra'] > 20:
        pontos += 1
    elif 10 <= row['valor_da_ultima_compra'] <= 20:
        pontos += 0.5
    elif row['valor_da_ultima_compra'] < 10:
        pontos += 0.25
    
    if row['trabalha'] == 'Sim':
        pontos += 1
    elif row['trabalha'] == 'Não':
        pontos += 0.5
    
    #pontos += row['interação_social']
    
    if row['faltas'] == 0:
        pontos += 1
    elif 1 <= row['faltas'] <= 2:
        pontos += 0.5
    elif row['faltas'] > 2:
        pontos -= 0.25
    
    return pontos

dados_pao_copia['pontos'] = dados_pao_copia.apply(classificar_cliente, axis=1)

# Seleção dos top 3 clientes
top_clientes = dados_pao_copia[['pessoa', 'pontos', 'pão_comprados']].sort_values(by='pontos', ascending=False).head(3)

# Calcular os valores mínimos e máximos de compra por dia
top_clientes['valor_minimo_dia'] = dados_pao_copia['valor_da_ultima_compra'] * 0.1  # Exemplo: 10% do valor da última compra
top_clientes['valor_maximo_dia'] = dados_pao_copia['valor_da_ultima_compra'] * 0.3  # Exemplo: 30% do valor da última compra

# Selecionar as colunas necessárias e salvar como CSV
resultado = top_clientes[['pessoa', 'pontos', 'valor_minimo_dia', 'valor_maximo_dia']]
resultado

#resultado.columns = ['Nome da Pessoa', 'Valor da possibilidade de compra por semana', 'Valor Minimo de compra de pão por dia', 'Valor Maximo de compra de pão por dia']
#resultado.to_csv("Probabilidade_pao.csv", index=False)

,pessoa,pontos,valor_minimo_dia,valor_maximo_dia
12,Thiago Ramos,3.75,0.5,1.5
35,Renata Ribeiro,3.75,0.1,0.3
32,Leonardo Ramos,3.75,0.6,1.8
